In [2]:
# Install required dependencies in Google Colab
import os
import sys
if 'google.colab' in sys.modules:
    !pip install rdkit
    !pip install vina
    !apt-get install openbabel
    !apt-get install autodock-vina
    !wget https://sourceforge.net/projects/openbabel/files/openbabel/3.1.1/openbabel-3.1.1.tar.gz
    !tar -xvzf openbabel-3.1.1.tar.gz
    !cd openbabel-3.1.1 && mkdir build && cd build && cmake .. && make && make install
    !wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64 -O /usr/local/bin/vina
    !chmod +x /usr/local/bin/vina


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 27.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libinchi1 libmaeparser1 libopenbabel7
The following NEW packages will be installed:
  libinchi1 libmaeparser1 libopenbabel7 openbabel
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 3,903 kB of archives.
After this operation, 16.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libinchi1 amd64 1.03+dfsg-4 [455 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmaeparser1 amd64 1.2.4-1build1 [88.2 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopenbabel7 amd64 3.1.1+dfsg-6ubuntu5 [3,231 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 openbabel amd64 3.1

In [3]:
# Import necessary libraries
from rdkit import Chem
from rdkit.Chem import AllChem
import subprocess
import os
from vina import Vina

# Define ligand data with SMILES and corresponding PDB IDs
ligands = {
    'METRIBUZIN': {'smiles': 'CSC1NNN(C(=O)N1N)C(C)(C)C', 'pdb_id': '7OUI'},
    'GLYPHOSATE': {'smiles': 'OC(=O)CNCP(=O)(O)O', 'pdb_id': '3FJZ'},
    '2,4-DICHLOROPHENOXYACETIC_ACID': {'smiles': 'OC(=O)COc1ccc(cc1Cl)Cl', 'pdb_id': '2P1N'},
    'Bispyribac-sodium': {'smiles': 'COc1nc(Oc2cccc(c2C(=O)[O-])Oc2nc(OC)cc(n2)OC)nc(c1)OC.[Na+]', 'pdb_id': '7U25'},
    'Herbicide_1': {'smiles': 'Clc1ccc(cc1)Oc1nnc(c(n1)S(=O)(=O)C)C(=O)[O-].[Na+]', 'pdb_id': ['7OUI', '3FJZ', '2P1N', '7U25']},
    'Herbicide_2': {'smiles': 'CCNN1C=NC(=NC1C(=S)[O-])Oc1ccc(cc1F)F.[K+]', 'pdb_id': ['7OUI', '3FJZ', '2P1N', '7U25']},
    'Herbicide_3': {'smiles': 'CNc1cc(nc(n1)S(=O)(=O)[NH-])C(F)(F)F.[NH4+]', 'pdb_id': ['7OUI', '3FJZ', '2P1N', '7U25']}
}

In [4]:
# Create directories for input and output files
os.makedirs('ligands', exist_ok=True)
os.makedirs('proteins', exist_ok=True)
os.makedirs('docking_results', exist_ok=True)

In [5]:
# Function to download PDB file
def download_pdb(pdb_id):
    """Download PDB file from RCSB PDB database."""
    subprocess.run(['wget', f'https://files.rcsb.org/download/{pdb_id}.pdb', '-O', f'proteins/{pdb_id}.pdb'])

In [6]:
# Function to prepare protein using Open Babel
def prepare_protein(pdb_id):
    """Prepare protein by removing water and converting to PDBQT format."""
    subprocess.run(['obabel', f'proteins/{pdb_id}.pdb', '-O', f'proteins/{pdb_id}_prepared.pdbqt', '-xr', '-h'])

In [11]:
# Function to convert SMILES to PDBQT
def smiles_to_pdbqt(smiles, ligand_name):
    """Convert SMILES to 3D structure and then to PDBQT format."""
    mol = Chem.MolFromSmiles(smiles)
    mol = neutralize_mol(mol)  # Neutralize the molecule
    mol = Chem.AddHs(mol)  # Add hydrogens
    AllChem.EmbedMolecule(mol, randomSeed=42)  # Generate 3D coordinates
    AllChem.UFFOptimizeMolecule(mol)  # Optimize geometry
    Chem.MolToPDBFile(mol, f'ligands/{ligand_name}.pdb')  # Save as PDB
    subprocess.run(['obabel', f'ligands/{ligand_name}.pdb', '-O', f'ligands/{ligand_name}.pdbqt', '-xh'])  # Convert to PDBQT

In [8]:
# Function to perform docking with AutoDock Vina
def dock_ligand(protein_pdbqt, ligand_pdbqt, ligand_name, pdb_id, center=(0, 0, 0), size=(20, 20, 20)):
    """Perform docking using AutoDock Vina."""
    v = Vina(sf_name='vina')
    v.set_receptor(protein_pdbqt)
    v.set_ligand_from_file(ligand_pdbqt)
    v.compute_vina_maps(center=center, box_size=size)
    v.dock(exhaustiveness=8)
    v.write_poses(f'docking_results/{ligand_name}_{pdb_id}_docked.pdbqt', n_poses=1, overwrite=True)

In [12]:
# Main execution
def main():
    # Download and prepare proteins
    pdb_ids = ['7OUI', '3FJZ', '2P1N', '7U25']
    for pdb_id in pdb_ids:
        print(f"Downloading and preparing protein {pdb_id}...")
        download_pdb(pdb_id)
        prepare_protein(pdb_id)

    # Process each ligand
    for ligand_name, data in ligands.items():
        print(f"Processing ligand {ligand_name}...")
        # Convert SMILES to PDBQT
        smiles_to_pdbqt(data['smiles'], ligand_name)

        # Determine target PDBs
        target_pdbs = data['pdb_id'] if isinstance(data['pdb_id'], list) else [data['pdb_id']]

        # Perform docking for each target protein
        for pdb_id in target_pdbs:
            print(f"Docking {ligand_name} to {pdb_id}...")
            dock_ligand(
                protein_pdbqt=f'proteins/{pdb_id}_prepared.pdbqt',
                ligand_pdbqt=f'ligands/{ligand_name}.pdbqt',
                ligand_name=ligand_name,
                pdb_id=pdb_id
            )

# Run the script
if __name__ == "__main__":
    main()

Processing ligand METRIBUZIN...
Docking METRIBUZIN to 7OUI...
Processing ligand GLYPHOSATE...
Docking GLYPHOSATE to 3FJZ...
Processing ligand 2,4-DICHLOROPHENOXYACETIC_ACID...
Docking 2,4-DICHLOROPHENOXYACETIC_ACID to 2P1N...
Processing ligand Bispyribac-sodium...
Docking Bispyribac-sodium to 7U25...
Processing ligand Herbicide_1...
Docking Herbicide_1 to 7OUI...
Docking Herbicide_1 to 3FJZ...
Docking Herbicide_1 to 2P1N...
Docking Herbicide_1 to 7U25...
Processing ligand Herbicide_2...
Docking Herbicide_2 to 7OUI...
Docking Herbicide_2 to 3FJZ...
Docking Herbicide_2 to 2P1N...
Docking Herbicide_2 to 7U25...
Processing ligand Herbicide_3...
Docking Herbicide_3 to 7OUI...
Docking Herbicide_3 to 3FJZ...
Docking Herbicide_3 to 2P1N...
Docking Herbicide_3 to 7U25...


In [10]:
from rdkit.Chem import SaltRemover

def neutralize_mol(mol):
    """Neutralize molecule by removing salts."""
    remover = SaltRemover.SaltRemover()
    return remover.StripMol(mol)

In [13]:
!zip -r results.zip ligands proteins docking_results

  adding: ligands/ (stored 0%)
  adding: ligands/METRIBUZIN.pdbqt (deflated 73%)
  adding: ligands/Herbicide_3.pdb (deflated 75%)
  adding: ligands/Bispyribac-sodium.pdbqt (deflated 74%)
  adding: ligands/METRIBUZIN.pdb (deflated 75%)
  adding: ligands/Herbicide_1.pdb (deflated 76%)
  adding: ligands/Herbicide_3.pdbqt (deflated 71%)
  adding: ligands/2,4-DICHLOROPHENOXYACETIC_ACID.pdbqt (deflated 70%)
  adding: ligands/Herbicide_2.pdb (deflated 76%)
  adding: ligands/Herbicide_2.pdbqt (deflated 72%)
  adding: ligands/Bispyribac-sodium.pdb (deflated 76%)
  adding: ligands/GLYPHOSATE.pdb (deflated 74%)
  adding: ligands/2,4-DICHLOROPHENOXYACETIC_ACID.pdb (deflated 75%)
  adding: ligands/GLYPHOSATE.pdbqt (deflated 71%)
  adding: ligands/Herbicide_1.pdbqt (deflated 71%)
  adding: proteins/ (stored 0%)
  adding: proteins/7OUI.pdb (deflated 81%)
  adding: proteins/3FJZ.pdb (deflated 76%)
  adding: proteins/2P1N.pdb (deflated 77%)
  adding: proteins/7U25_prepared.pdbqt (deflated 78%)
  adding

In [14]:
from google.colab import files
files.download('results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>